In [96]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from scipy.optimize import curve_fit

from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import matplotlib.ticker as mticker

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

In [97]:
# Frequenze suggerite:
pts = 35
hint_nu = np.logspace(1,7, pts, base=10)
print(hint_nu)

[1.00000000e+01 1.50131073e+01 2.25393390e+01 3.38385515e+01
 5.08021805e+01 7.62698586e+01 1.14504757e+02 1.71907220e+02
 2.58086154e+02 3.87467512e+02 5.81709133e+02 8.73326162e+02
 1.31113394e+03 1.96841945e+03 2.95520924e+03 4.43668733e+03
 6.66084629e+03 1.00000000e+04 1.50131073e+04 2.25393390e+04
 3.38385515e+04 5.08021805e+04 7.62698586e+04 1.14504757e+05
 1.71907220e+05 2.58086154e+05 3.87467512e+05 5.81709133e+05
 8.73326162e+05 1.31113394e+06 1.96841945e+06 2.95520924e+06
 4.43668733e+06 6.66084629e+06 1.00000000e+07]


In [98]:
nu = np.array([10.22,19.85,42.65,88.3,105.7,129.9,183.7,378,784,1623,3325,6920,14050,29138,61650,127050,258300,349000,545500,1033500,2608000,4805000,10810000])
V_in = np.array([810,820,820,820,820,820,810,810,810,810,810,810,810,810,800,800,800,810,800,800,800,790,740])*10**-3
V_out = np.array([0.56,1.05,2.05,3.64,4.10,4.55,5.23,5.99,6.27,6.35,6.35,6.35,6.31,6.20,6.07,5.71,4.70,4.02,3.02,1.81,0.8,0.44,0.24])

In [99]:
sigmaV = lambda a: a*0.03

In [100]:
def tabmaker(l):
    st = ""
    for i in range(len(l[0][1])):
        for j in range(len(l)):
            col = l[j]
            #print(col)
            st+="${}\pm{}$ & ".format(np.round(col[1][i], 4), np.round(col[0](col[1][i]), 2))
        st += " \\\ "
    return st

In [101]:
print(tabmaker([(sigmaV, V_in), (sigmaV, V_out)]))

$0.81\pm0.02$ & $0.56\pm0.02$ &  \\ $0.82\pm0.02$ & $1.05\pm0.03$ &  \\ $0.82\pm0.02$ & $2.05\pm0.06$ &  \\ $0.82\pm0.02$ & $3.64\pm0.11$ &  \\ $0.82\pm0.02$ & $4.1\pm0.12$ &  \\ $0.82\pm0.02$ & $4.55\pm0.14$ &  \\ $0.81\pm0.02$ & $5.23\pm0.16$ &  \\ $0.81\pm0.02$ & $5.99\pm0.18$ &  \\ $0.81\pm0.02$ & $6.27\pm0.19$ &  \\ $0.81\pm0.02$ & $6.35\pm0.19$ &  \\ $0.81\pm0.02$ & $6.35\pm0.19$ &  \\ $0.81\pm0.02$ & $6.35\pm0.19$ &  \\ $0.81\pm0.02$ & $6.31\pm0.19$ &  \\ $0.81\pm0.02$ & $6.2\pm0.19$ &  \\ $0.8\pm0.02$ & $6.07\pm0.18$ &  \\ $0.8\pm0.02$ & $5.71\pm0.17$ &  \\ $0.8\pm0.02$ & $4.7\pm0.14$ &  \\ $0.81\pm0.02$ & $4.02\pm0.12$ &  \\ $0.8\pm0.02$ & $3.02\pm0.09$ &  \\ $0.8\pm0.02$ & $1.81\pm0.05$ &  \\ $0.8\pm0.02$ & $0.8\pm0.02$ &  \\ $0.79\pm0.02$ & $0.44\pm0.01$ &  \\ $0.74\pm0.02$ & $0.24\pm0.01$ &  \\ 
